In [160]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time
driver= webdriver.Chrome()
driver.get("https://instagram.com/")

In [161]:
# Select the password & username
username=WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[name='username']")))
password=WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[name='password']")))

username.clear()
password.clear()

#Insert login info 
#Please Enter your account for the script to run 
username.send_keys("myusername")
password.send_keys("mypassword")


In [162]:
#login
login=WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"button[type='submit']"))).click()    

In [163]:
# pass the first 2 pop ups after the login
not_now1=WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"//button[contains(text(),'Not Now')]"))).click()
not_now2=WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"//button[contains(text(),'Not Now')]"))).click()

In [164]:
#Activate the searchbox 
activate_search=WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"div[class='xh8yej3 x1iyjqo2']>div:nth-of-type(2)"))).click()

#Select the searchbox
searchbox=WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"//input[@placeholder='Search']"))) 

searchbox.clear()

keyword="#bmwm5"

#search for the keyword
searchbox.send_keys(keyword)

# Wait for 5 seconds
time.sleep(5) 

#Enter to the keyword content 
key_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, '/" + keyword[1:] + "/')]")))
key_link.click()

In [165]:
#searchbox.send_keys(Keys.ENTER)

In [166]:
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Wait to load page 
    time.sleep(10)
    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [167]:
#target all the link elements on the page
post_links = driver.find_elements(By.TAG_NAME, "a")
post_links = [a.get_attribute('href') for a in post_links]

#narrow down all links to image links only
post_links = [a for a in post_links if str(a).startswith("https://www.instagram.com/p/")]

print('Found ' + str(len(post_links)) + ' links to images')
post_links[:5]

Found 45 links to images


['https://www.instagram.com/p/ClLKkROqGyE/',
 'https://www.instagram.com/p/ClJW6OjO_NQ/',
 'https://www.instagram.com/p/ClNq4ecqkAr/',
 'https://www.instagram.com/p/ClEwV0gKwK-/',
 'https://www.instagram.com/p/ClO2AyUKryp/']

In [168]:
from selenium.common.exceptions import TimeoutException
def tap(wait,xpath):
        element = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
        element.click()
    

In [169]:
images = []
data=[]
wait = WebDriverWait(driver, 1)
i=0
#Follow each post link to all post images, text, and comments
for link in post_links[:40]:
    driver.get(link)
    time.sleep(5)
    #Tap until the end to open all collapsed elements
    while True:
        try:
           tap(wait,"//button[class='_acan _acao _acas']")
           tap(wait,"//button[class='_abl']")    
        except TimeoutException:
            break
    #Tap until the end to get all images
    while True:
        try:
            
            img=driver.find_elements(By.XPATH, "//div[@class='_aagu _aato']//img")
            img = [i.get_attribute('src') for i in img]
            tap(wait,"//button[@aria-label='Next']")
        except TimeoutException:
            break
    img = list(dict.fromkeys(img))    
    print(type(img))
    text= driver.find_elements(By.XPATH, "//div[@class='_ae5q _ae5r _ae5s']//span[text()]")
    text = [t.text for t in text]
    print(text[0])
    
    print(len(img))
    post={"_id": i,"post_link":link,"caption":text[0],"comments":text[1:]}
    i=i+1
    data.append(post)
   
      

<class 'list'>
Owner : @dikkntouring_e34 .
.
.
.
.
.
#e34 #e34empire #bmwe34 #e34m5 #stancenation #oldstyle #bmwm #m5 #bmw #bmwmotorsport #mpower #ultimatestance #bimmer #e34kingdom #germancars #bmwfamily #bmwrepost #bmwnation #ahmetbiroglu #autodisiac #bmwpower #bmwclub #oldschool #bmwm5 #bmwlife #bmwmpower #bmwclassic #bmwgram #bmwpost #bmwfan
0
<class 'list'>
2023 M8 Gran Coupe Competition in Marina Bay Blue 💙 Contact us for more information.

☎️ 973-988-1840
🌎 paulmillerbmw.com
📍 Paul Miller BMW
1515 NJ-23
Wayne, NJ 07470

@paulmillerbmw • @paulmillerautogroup
0
<class 'list'>
⭕️🏆⭕️ Lovely 528i from Czech Republic ✅ 193hp petrol engine 🚀🚀 Manual gearbox 🕹️ 19" OEM wheels Style 95 ⭕️ M-paket bumper 😎 Xenon headlights & angel eyes 🔆 Congrats to the proud owner @patrik_sauman_e39 ‼️ Any comments ? 👀 Rate it if you like 🚗👍👍👍👍 #e39 #e39mafia #e39m5 #e39touring #e39fanatics #e39club #e39world #bmwe39 #m5e39 #bmwm5e39 #e39love #bmw #bmwm #bmwlove #bmwm5 #bmwlife #bmwclub #bmwmpower #bmwfa

In [179]:
import pymongo
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["IG_Scraping"]
collection=mydb["posts"]

In [197]:
result = collection.insert_many(data)
result

In [198]:
for c in collection.find().limit(5):
  print(c)

{'_id': 0, 'post_link': 'https://www.instagram.com/p/ClLKkROqGyE/', 'caption': 'Owner : @dikkntouring_e34 .\n.\n.\n.\n.\n.\n#e34 #e34empire #bmwe34 #e34m5 #stancenation #oldstyle #bmwm #m5 #bmw #bmwmotorsport #mpower #ultimatestance #bimmer #e34kingdom #germancars #bmwfamily #bmwrepost #bmwnation #ahmetbiroglu #autodisiac #bmwpower #bmwclub #oldschool #bmwm5 #bmwlife #bmwmpower #bmwclassic #bmwgram #bmwpost #bmwfan', 'comments': ['Where can I get an e34 empire badge please', 'View replies (2)', 'What’s the name of that wheel?', 'Thanks @ahmetbiroglu 💪🏼', 'View replies (1)']}
{'_id': 1, 'post_link': 'https://www.instagram.com/p/ClJW6OjO_NQ/', 'caption': '2023 M8 Gran Coupe Competition in Marina Bay Blue 💙 Contact us for more information.\n\n☎️ 973-988-1840\n🌎 paulmillerbmw.com\n📍 Paul Miller BMW\n1515 NJ-23\nWayne, NJ 07470\n\n@paulmillerbmw • @paulmillerautogroup', 'comments': ['Verified', 'Oof what a color!', 'View replies (1)', '#eleanor_officialls_ honestly receiving $5,350 after in